# Week 12: Customer Lifetime Value & Final Capstone

**Goal:** Master Customer Lifetime Value analysis and complete a comprehensive marketing measurement capstone project.

**Time Commitment:** ~1 hour per day × 7 days = 7 hours total

**What You'll Learn:**
- Customer Lifetime Value (CLV/LTV) fundamentals
- Historical CLV calculation
- Predictive CLV modeling
- Cohort-based analysis
- LTV:CAC ratio optimization
- CLV-based budget allocation
- Integrating all measurement techniques

**Why This Matters:**
As a Marketing Measurement Partner, CLV is crucial for:
- Understanding true customer value over time
- Justifying customer acquisition costs
- Optimizing for long-term profitability, not just conversions
- Segmenting customers by value
- Making retention vs acquisition trade-offs
- Calculating allowable CAC (Customer Acquisition Cost)

CLV shifts focus from individual transactions to customer relationships. It's the ultimate measure of marketing success.

This is your final week - let's make it count! 🚀

---

## 📅 Day 78: Customer Lifetime Value Fundamentals (~60 min)

### Learning Objectives
- Understand what CLV is and why it matters
- Learn the components of CLV
- Calculate simple CLV metrics
- Understand CLV vs CAC relationship

### The Business Problem
You spend $50 to acquire a customer. Is that good or bad? It depends on how much revenue that customer generates over their lifetime. You need to calculate CLV to know if your acquisition strategy is profitable.

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import warnings
warnings.filterwarnings('ignore')

# Settings
np.random.seed(42)
sns.set_style('whitegrid')
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 2)

### 📖 Concept: What is Customer Lifetime Value?

**CLV (Customer Lifetime Value)** = The total net profit a customer generates over their entire relationship with your business.

**Simple Formula:**
```
CLV = (Average Order Value) × (Purchase Frequency) × (Customer Lifespan)
```

**More Sophisticated:**
```
CLV = Σ [(Profit per period t) × (Retention rate^t) / (1 + discount rate)^t]
```

**Components:**
1. **Revenue per customer** (orders × AOV)
2. **Gross margin** (revenue - COGS)
3. **Retention rate** (% who stay active)
4. **Discount rate** (time value of money)
5. **Churn rate** (opposite of retention)

### 📖 Concept: CLV vs CAC

**CAC (Customer Acquisition Cost)** = Total marketing + sales costs / New customers acquired

**Key Metric: LTV:CAC Ratio**
- **< 1**: Losing money on every customer (unsustainable)
- **1-3**: Not enough margin (may be profitable but risky)
- **3-5**: Healthy ratio (good business)
- **> 5**: May be under-investing in growth

**Goal:** LTV:CAC ≥ 3.0

In [ ]:
# Simple CLV calculation
def calculate_simple_clv(avg_order_value, purchase_frequency_per_year, 
                         avg_customer_lifespan_years, gross_margin=0.50):
    """
    Calculate simple CLV.
    
    Parameters:
    - avg_order_value: Average $ per order
    - purchase_frequency_per_year: How many orders per year
    - avg_customer_lifespan_years: How many years customer stays active
    - gross_margin: Profit margin (default 50%)
    
    Returns:
    - CLV in dollars
    """
    annual_revenue_per_customer = avg_order_value * purchase_frequency_per_year
    total_revenue = annual_revenue_per_customer * avg_customer_lifespan_years
    clv = total_revenue * gross_margin
    
    return clv

# Example: E-commerce business
clv = calculate_simple_clv(
    avg_order_value=75,
    purchase_frequency_per_year=4,
    avg_customer_lifespan_years=3,
    gross_margin=0.50
)

print("Simple CLV Calculation")
print("=" * 60)
print(f"Average Order Value: $75")
print(f"Purchase Frequency: 4 times/year")
print(f"Customer Lifespan: 3 years")
print(f"Gross Margin: 50%")
print(f"\nCalculated CLV: ${clv:.2f}")

# Compare to CAC
cac = 50
ltv_cac_ratio = clv / cac

print(f"\nIf CAC = ${cac}:")
print(f"LTV:CAC Ratio = {ltv_cac_ratio:.2f}x")

if ltv_cac_ratio >= 3:
    print("✅ Healthy ratio - good unit economics")
elif ltv_cac_ratio >= 1:
    print("⚠️  Profitable but margin is tight")
else:
    print("❌ Losing money - need to reduce CAC or increase CLV")

### 💡 Try It: Calculate CLV for Different Segments

Different customer segments have different CLV profiles.

In [ ]:
# YOUR CODE HERE
# Calculate CLV for three segments:
#
# Segment 1 - High Value:
# - AOV: $150, Frequency: 6/year, Lifespan: 5 years, Margin: 55%
#
# Segment 2 - Medium Value:
# - AOV: $75, Frequency: 4/year, Lifespan: 3 years, Margin: 50%
#
# Segment 3 - Low Value:
# - AOV: $40, Frequency: 2/year, Lifespan: 1.5 years, Margin: 45%
#
# Questions:
# 1. Calculate CLV for each segment
# 2. If CAC is $50 for all, what's the LTV:CAC ratio for each?
# 3. How much more can you pay to acquire high-value customers?
# 4. Should you focus on acquiring all segments equally?



### 📖 Concept: Generating Customer Transaction Data

Let's create a realistic customer dataset to work with.

In [ ]:
# Generate synthetic customer transaction data
def generate_customer_data(n_customers=5000):
    """
    Generate realistic customer transaction data.
    """
    np.random.seed(42)
    
    customers = []
    transactions = []
    
    start_date = datetime(2022, 1, 1)
    end_date = datetime(2024, 11, 16)
    
    for customer_id in range(n_customers):
        # Customer acquisition date
        acquisition_date = start_date + timedelta(days=np.random.randint(0, 700))
        
        # Customer segment (high, medium, low value)
        segment = np.random.choice(['high', 'medium', 'low'], p=[0.20, 0.50, 0.30])
        
        # Segment characteristics
        if segment == 'high':
            aov = np.random.normal(150, 30)
            annual_frequency = np.random.poisson(6)
            churn_rate_monthly = 0.02  # Low churn
        elif segment == 'medium':
            aov = np.random.normal(75, 15)
            annual_frequency = np.random.poisson(4)
            churn_rate_monthly = 0.05
        else:  # low
            aov = np.random.normal(40, 10)
            annual_frequency = np.random.poisson(2)
            churn_rate_monthly = 0.10  # High churn
        
        # Customer record
        customers.append({
            'customer_id': customer_id,
            'acquisition_date': acquisition_date,
            'segment': segment
        })
        
        # Generate transactions
        is_active = True
        current_date = acquisition_date
        transaction_num = 0
        
        while is_active and current_date < end_date:
            # Create transaction
            order_value = max(aov + np.random.normal(0, aov * 0.2), 10)
            
            transactions.append({
                'customer_id': customer_id,
                'transaction_date': current_date,
                'order_value': order_value,
                'transaction_num': transaction_num
            })
            
            transaction_num += 1
            
            # Next purchase (days until next)
            days_to_next = max(int(np.random.exponential(365 / annual_frequency)), 7)
            current_date += timedelta(days=days_to_next)
            
            # Check if customer churns
            if np.random.random() < churn_rate_monthly:
                is_active = False
    
    df_customers = pd.DataFrame(customers)
    df_transactions = pd.DataFrame(transactions)
    
    return df_customers, df_transactions

# Generate data
df_customers, df_transactions = generate_customer_data()

print("Customer Dataset:")
print(f"Total customers: {len(df_customers):,}")
print(f"Total transactions: {len(df_transactions):,}")
print(f"\nCustomer segments:")
print(df_customers['segment'].value_counts())
print("\nSample customers:")
print(df_customers.head())
print("\nSample transactions:")
print(df_transactions.head())

### ✏️ Exercise 1: Calculate Historical CLV by Segment

Using the transaction data, calculate actual CLV by segment.

In [ ]:
# YOUR CODE HERE
# For each customer:
# 1. Calculate total revenue (sum of all transactions)
# 2. Calculate total profit (revenue × 50% margin)
# 3. Count number of transactions
# 4. Calculate customer lifespan (days from first to last transaction)
#
# Then aggregate by segment:
# 5. Average CLV per segment
# 6. Average number of transactions per segment
# 7. Average lifespan per segment
# 8. Compare to simple CLV calculations - how close were they?



### 🎯 Day 78 Mini-Project: CLV Dashboard

Build a comprehensive CLV analysis dashboard.

In [ ]:
# YOUR CODE HERE
# Create a CLV dashboard that shows:
#
# 1. Overall metrics:
#    - Total customers
#    - Average CLV
#    - Total LTV (sum of all customer values)
#
# 2. Segment comparison:
#    - CLV by segment (table and bar chart)
#    - Distribution of CLV (histogram)
#    - Segment mix (pie chart)
#
# 3. Time analysis:
#    - CLV by acquisition cohort (month)
#    - Customer retention curves by segment
#
# 4. Business insights:
#    - If CAC is $50, which segments are profitable?
#    - What % of total LTV comes from each segment?
#    - Recommendations for acquisition strategy



### 🎓 Day 78 Key Takeaways

✅ CLV measures total customer value over lifetime  
✅ LTV:CAC ratio is critical for profitability  
✅ Different segments have vastly different CLV  
✅ Historical CLV is calculated from actual data  
✅ CLV informs acquisition and retention strategy  

**Next:** Tomorrow we'll build predictive CLV models!

---

## 📅 Day 79: Historical CLV Calculation (~60 min)

### Learning Objectives
- Calculate historical CLV from transaction data
- Understand cohort analysis
- Build retention curves
- Measure customer health metrics

### The Business Problem
You need to understand which customer cohorts are most valuable and track how CLV changes over time to inform acquisition strategy.

### 📖 Concept: Cohort Analysis

**Cohort** = Group of customers acquired in the same time period (month, quarter, etc.)

**Why Cohorts Matter:**
- Compare acquisition quality over time
- Track retention improvements
- Identify seasonal patterns
- Measure impact of product/marketing changes

In [ ]:
# Prepare cohort data
df_customers['cohort_month'] = pd.to_datetime(df_customers['acquisition_date']).dt.to_period('M')

# Merge customers with transactions
df_merged = df_transactions.merge(df_customers[['customer_id', 'cohort_month', 'segment']], 
                                   on='customer_id')

# Calculate customer-level metrics
customer_metrics = df_transactions.groupby('customer_id').agg({
    'order_value': ['sum', 'mean', 'count'],
    'transaction_date': ['min', 'max']
}).reset_index()

customer_metrics.columns = ['customer_id', 'total_revenue', 'avg_order_value', 
                             'num_orders', 'first_purchase', 'last_purchase']

# Calculate lifespan in days
customer_metrics['lifespan_days'] = (
    customer_metrics['last_purchase'] - customer_metrics['first_purchase']
).dt.days

# Calculate CLV (assuming 50% margin)
customer_metrics['clv'] = customer_metrics['total_revenue'] * 0.50

# Merge back with cohort info
customer_metrics = customer_metrics.merge(
    df_customers[['customer_id', 'cohort_month', 'segment']], 
    on='customer_id'
)

print("Customer Metrics Summary:")
print(customer_metrics.describe())
print("\nCLV by Segment:")
print(customer_metrics.groupby('segment')['clv'].describe())

### 💡 Try It: Cohort CLV Analysis

Analyze CLV trends by acquisition cohort.

In [ ]:
# YOUR CODE HERE
# 1. Calculate average CLV by cohort month
# 2. Plot CLV over time (are newer cohorts better or worse?)
# 3. Calculate the trend - is CLV improving?
# 4. Which cohort has the highest CLV? Why might that be?
# 5. For recent cohorts (last 6 months), CLV might be understated - why?



### 📖 Concept: Retention Curves

Retention curves show what % of customers remain active over time.

In [ ]:
# Calculate months since acquisition for each transaction
df_merged['transaction_month'] = pd.to_datetime(df_merged['transaction_date']).dt.to_period('M')
df_merged['months_since_acquisition'] = (
    (df_merged['transaction_month'] - df_merged['cohort_month']).apply(lambda x: x.n)
)

# Create retention table
# For each cohort, count active customers in each month
retention_data = []

for cohort in df_merged['cohort_month'].unique():
    cohort_customers = df_merged[df_merged['cohort_month'] == cohort]['customer_id'].unique()
    cohort_size = len(cohort_customers)
    
    for month in range(0, 24):  # First 24 months
        active_customers = df_merged[
            (df_merged['cohort_month'] == cohort) & 
            (df_merged['months_since_acquisition'] == month)
        ]['customer_id'].nunique()
        
        retention_rate = active_customers / cohort_size if cohort_size > 0 else 0
        
        retention_data.append({
            'cohort': cohort,
            'month': month,
            'active_customers': active_customers,
            'cohort_size': cohort_size,
            'retention_rate': retention_rate
        })

df_retention = pd.DataFrame(retention_data)

# Calculate average retention curve across all cohorts
avg_retention = df_retention.groupby('month')['retention_rate'].mean().reset_index()

# Plot
plt.figure(figsize=(12, 6))
plt.plot(avg_retention['month'], avg_retention['retention_rate'] * 100, 
         marker='o', linewidth=2, markersize=6)
plt.xlabel('Months Since Acquisition')
plt.ylabel('Retention Rate (%)')
plt.title('Customer Retention Curve (Average Across All Cohorts)')
plt.grid(True, alpha=0.3)
plt.show()

print(f"Month 0 Retention: {avg_retention[avg_retention['month']==0]['retention_rate'].values[0]*100:.1f}%")
print(f"Month 12 Retention: {avg_retention[avg_retention['month']==12]['retention_rate'].values[0]*100:.1f}%")

### ✏️ Exercise 2: Segment Retention Comparison

Compare retention curves across customer segments.

In [ ]:
# YOUR CODE HERE
# 1. Calculate retention curves by segment (high, medium, low)
# 2. Plot all three on the same chart
# 3. Which segment has the best retention?
# 4. At month 12, what's the retention rate difference?
# 5. How does this relate to CLV differences?



### 🎯 Day 79 Mini-Project: Cohort Report Generator

Build an automated cohort analysis report.

In [ ]:
# YOUR CODE HERE
# Create a comprehensive cohort analysis that includes:
#
# 1. Cohort summary table:
#    - Cohort month
#    - Number of customers
#    - Average CLV
#    - Average orders
#    - Average lifespan
#
# 2. Retention heatmap:
#    - Rows: Cohorts
#    - Columns: Months since acquisition
#    - Values: Retention %
#
# 3. CLV progression:
#    - How CLV grows month-by-month for each cohort
#
# 4. Business insights:
#    - Are cohorts improving over time?
#    - When does most CLV happen (early or late)?
#    - Recommendations for retention strategy



### 🎓 Day 79 Key Takeaways

✅ Cohort analysis tracks customer value over time  
✅ Retention curves reveal churn patterns  
✅ Different segments have different retention profiles  
✅ Historical CLV is built from transaction data  
✅ Cohort insights drive acquisition and retention decisions  

**Next:** Tomorrow we'll build predictive CLV models!

---

## 📅 Day 80-82: Advanced CLV Topics (Condensed)

### Day 80: Predictive CLV Modeling
- Using machine learning to predict future CLV
- Feature engineering from early behavior
- Random Forest and XGBoost models
- Predicting CLV after first purchase

### Day 81: Cohort-Based CLV
- Cohort retention modeling
- Revenue curves by cohort
- Lifetime value projections
- Comparing cohort performance

### Day 82: LTV:CAC Ratio Analysis
- Calculating CAC by channel
- Segment-specific LTV:CAC ratios
- Payback period analysis
- Optimizing acquisition strategy
- Channel budget allocation based on LTV:CAC

*Note: These sections would be fully expanded in a production version with detailed code examples and exercises.*

---

## 📅 Day 83-84: FINAL CAPSTONE PROJECT (~2 hours)

### 🎯 The Ultimate Marketing Measurement Challenge

**Congratulations!** You've reached the final capstone. This project integrates **everything** you've learned:
- Python and Pandas fundamentals
- Statistical analysis and A/B testing
- Attribution modeling
- Marketing Mix Modeling
- Incrementality testing
- Customer Lifetime Value

### The Scenario

You're the newly hired **Marketing Measurement Partner** at **GrowthCo**, a fast-growing e-commerce company. The CMO has tasked you with a comprehensive marketing audit and optimization plan.

**Your Mission:**
Analyze GrowthCo's marketing performance across all channels, prove incrementality, optimize budget allocation, and present recommendations to the executive team.

**Available Data:**
1. Customer transaction data (2 years)
2. Marketing spend by channel (weekly)
3. Customer journey touchpoints
4. Recent A/B test results
5. Geo experiment data

**Your Deliverables:**
1. Executive summary
2. Attribution analysis
3. Marketing Mix Model
4. Incrementality findings
5. CLV and LTV:CAC analysis
6. Budget optimization recommendations
7. Measurement roadmap for next quarter

### Part 1: Data Preparation & Exploration (20 min)

Load and understand the data.

In [ ]:
# YOUR CODE HERE
# 
# Use the customer and transaction data you generated earlier, plus:
# 
# 1. Generate weekly marketing spend data (similar to Week 10)
#    - Channels: Google Search, Facebook, Instagram, Display, Email
#    - 104 weeks of data
#    - Include weekly sales/conversions
#
# 2. Generate customer journey data (similar to Week 9)
#    - Multi-touch journeys for 1000 conversions
#    - Channels: Same as above
#
# 3. Create a summary of available data:
#    - Total customers
#    - Total revenue
#    - Time periods covered
#    - Marketing channels



### Part 2: Attribution Analysis (25 min)

Compare multiple attribution models.

In [ ]:
# YOUR CODE HERE
#
# Build and compare attribution models:
# 1. Last-touch attribution
# 2. First-touch attribution  
# 3. Linear attribution
# 4. Position-based (40-20-40)
#
# For each model:
# - Calculate attributed conversions by channel
# - Calculate ROAS by channel (use actual spend data)
#
# Create comparison table and visualization:
# - How do results differ across models?
# - Which channels benefit from multi-touch attribution?
# - Recommendations for budget allocation



### Part 3: Marketing Mix Model (25 min)

Build an MMM to measure aggregate channel impact.

In [ ]:
# YOUR CODE HERE
#
# Build a Marketing Mix Model:
# 1. Prepare weekly data with control variables
#    - Trend, seasonality, holidays
#
# 2. Apply transformations:
#    - Adstock (geometric decay)
#    - Saturation (log or square root)
#
# 3. Build regression model
#    - Sales ~ marketing channels + controls
#
# 4. Validate model:
#    - R² score
#    - Residual diagnostics
#
# 5. Extract insights:
#    - Coefficient interpretation
#    - Marginal ROI by channel
#    - Contribution to total sales
#
# 6. Compare MMM results to attribution results



### Part 4: Incrementality Analysis (20 min)

Analyze incrementality test results.

In [ ]:
# YOUR CODE HERE
#
# Scenario: You ran a geo test on Facebook advertising
#
# 1. Generate geo test data:
#    - 20 test markets (increased Facebook spend 50%)
#    - 20 control markets (no change)
#    - 8 weeks pre-test, 8 weeks test period
#    - True incremental lift: 8%
#
# 2. Analyze using difference-in-differences:
#    - Calculate pre/post change in test vs control
#    - Statistical significance
#
# 3. Calculate incrementality metrics:
#    - Incremental conversions
#    - Incremental ROAS
#    - Incrementality rate
#
# 4. Compare to attribution and MMM estimates:
#    - Does Facebook show similar ROI across methods?
#    - Which method is most conservative?



### Part 5: CLV & LTV:CAC Analysis (20 min)

Analyze customer lifetime value and unit economics.

In [ ]:
# YOUR CODE HERE
#
# Using customer transaction data:
# 1. Calculate CLV by segment and channel
#    - Which acquisition channels bring highest LTV customers?
#
# 2. Calculate CAC by channel
#    - Total spend / new customers acquired
#
# 3. Calculate LTV:CAC ratios
#    - Overall and by channel
#    - Which channels have healthy ratios (>3)?
#
# 4. Payback period analysis
#    - How long to recover CAC?
#    - Varies by channel?
#
# 5. Create CLV-based budget recommendations
#    - Shift budget to channels with best LTV:CAC



### Part 6: Budget Optimization (15 min)

Create data-driven budget allocation recommendations.

In [ ]:
# YOUR CODE HERE
#
# Synthesize all analyses:
# 1. Create a scoring framework:
#    - Attribution ROAS (30%)
#    - MMM marginal ROI (30%)
#    - Incrementality ROAS (25%)
#    - LTV:CAC ratio (15%)
#
# 2. Rank channels by composite score
#
# 3. Current budget allocation: [Your choice]
#
# 4. Proposed new allocation:
#    - Based on composite scores
#    - Constraints: Min/max per channel
#
# 5. Expected impact:
#    - Incremental conversions
#    - Incremental revenue
#    - ROI improvement



### Part 7: Executive Summary & Recommendations (15 min)

Create an executive-ready summary and roadmap.

In [ ]:
# YOUR CODE HERE
#
# Create a structured executive summary:
#
# 1. SITUATION
#    - Current marketing performance
#    - Key metrics (ROAS, LTV:CAC, etc.)
#
# 2. INSIGHTS
#    - Top 3 findings from your analysis
#    - Supporting data/visualizations
#
# 3. RECOMMENDATIONS
#    - Immediate actions (next 30 days)
#    - Strategic changes (next quarter)
#    - Budget reallocation proposal
#
# 4. EXPECTED IMPACT
#    - Revenue lift
#    - ROI improvement
#    - Efficiency gains
#
# 5. MEASUREMENT ROADMAP
#    - Tests to run next quarter
#    - Infrastructure to build
#    - Reporting improvements
#
# Make it visual, concise, and actionable!



### Bonus Challenge (Optional)

If you complete the capstone early, try these advanced challenges:

1. **Predictive CLV Model**: Build a machine learning model to predict CLV from first purchase
2. **Channel Synergy Analysis**: Do some channels work better together? (interaction effects)
3. **Sequential Attribution**: Analyze conversion paths and identify key sequences
4. **Real-Time Dashboard**: Create an interactive dashboard using Plotly or Streamlit
5. **Monte Carlo Simulation**: Simulate budget scenarios with uncertainty
6. **Causal Impact Analysis**: Use Bayesian structural time series for impact estimation

---

## 🎓 CONGRATULATIONS! 🎉

### You've Completed the Marketing Measurement Partner Academy!

**What You've Mastered:**

**Weeks 1-2: Foundations**
- ✅ Python fundamentals
- ✅ Pandas data manipulation
- ✅ Marketing metrics calculation

**Week 8: A/B Testing**
- ✅ Hypothesis testing
- ✅ Sample size calculations
- ✅ Statistical significance
- ✅ Business impact analysis

**Week 9: Attribution**
- ✅ Multi-touch attribution models
- ✅ Customer journey analysis
- ✅ Rule-based and data-driven attribution

**Week 10: Marketing Mix Modeling**
- ✅ Regression modeling for marketing
- ✅ Adstock and saturation
- ✅ Budget optimization

**Week 11: Incrementality**
- ✅ Causal inference
- ✅ Geo experiments
- ✅ Difference-in-differences

**Week 12: CLV**
- ✅ Customer lifetime value
- ✅ Cohort analysis
- ✅ LTV:CAC optimization

### You Are Now a Marketing Measurement Partner!

You have the skills to:
- 📊 Analyze marketing performance rigorously
- 🧪 Design and analyze experiments
- 📈 Build measurement models
- 💰 Optimize budgets with data
- 🎯 Prove incrementality and causality
- 💎 Measure customer lifetime value
- 🚀 Drive real business impact

### What's Next?

1. **Apply these skills** to real marketing data at your company
2. **Share your knowledge** - teach others what you've learned
3. **Keep learning** - explore advanced topics like:
   - Bayesian methods
   - Machine learning for marketing
   - Automated bidding strategies
   - Real-time optimization
4. **Build your portfolio** - showcase your projects on GitHub
5. **Join the community** - connect with other measurement professionals

### Final Words

Marketing measurement is both **science and art**. You now have the science. The art comes from:
- Asking the right questions
- Understanding business context
- Communicating insights clearly
- Making decisions under uncertainty
- Balancing rigor with pragmatism

**You're ready. Go make an impact.** 🚀

---

*Thank you for completing the Marketing Measurement Partner Academy. We can't wait to see what you build!*